In [2]:
import pandas as pd
import numpy as np

address_clast = pd.read_csv('/Users/artyomkholodkov/Downloads/test/address_clust.csv')
address_stats = pd.read_csv('/Users/artyomkholodkov/Downloads/test/address_stats.csv')

In [41]:
address_clast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6996 entries, 0 to 6995
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   address_id  6996 non-null   int64
 1   cluster_id  6996 non-null   int64
dtypes: int64(2)
memory usage: 109.4 KB


In [42]:
address_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76724 entries, 0 to 76723
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              76724 non-null  int64  
 1   address_id      76724 non-null  int64  
 2   transaction_id  76724 non-null  int64  
 3   received        76724 non-null  float64
 4   sent            76724 non-null  float64
 5   cluster         76724 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 3.5 MB


Фунцкия, соотносящая address_id с кластером

In [10]:
def raw_func(raw):
    try:
        cluster = address_clast.loc[address_clast['address_id'] == raw['address_id']]['cluster_id'].values[0]
        return cluster
    except IndexError:
        return 0

In [11]:
address_stats['cluster'] = address_stats.apply(raw_func, axis=1)

In [12]:
address_stats

,id,address_id,transaction_id,received,sent,cluster
0,87134765,46402336,19162323,0.0,1800000.0,0
1,87134766,45919235,19162323,0.0,1071052.0,0
2,87134767,46529090,19162323,1000000.0,0.0,1
3,87134768,46529091,19162323,1821052.0,0.0,0
4,87154706,46529090,19166856,0.0,1000000.0,1
...,...,...,...,...,...,...
76719,139448796,60563257,30053430,4597692.0,0.0,0
76720,139449242,60563257,30053558,0.0,4597692.0,0
76721,139449243,60563257,30053558,0.0,16270.0,0
76722,139449244,49604329,30053558,2100000.0,0.0,1


Найдем количество уникальных transaction_id в датасете

In [29]:
unique_transaction_ids = np.unique(address_stats['transaction_id'].values)

Функция, вычисляющая cash flow. Имеет в виду, что отправка производистя из одного кластера.

In [30]:
def finf_flow(from_to: tuple):
    s = 0
    
    for unq_id in unique_transaction_ids:
        frame = address_stats[address_stats['transaction_id'] == unq_id]
        if from_to[0] in frame[frame['sent'] > 0]['cluster'].values and from_to[1] in frame[frame['received'] > 0]['cluster'].values:
            s += frame[frame['cluster'] == from_to[1]]['received'].sum() / 10**8
            
    print('кластер {} отправил {} биткойнов в кластер {}'.format(from_to[0], s, from_to[1]))

1

Находим cash flows

In [31]:
finf_flow(from_to = (1, 2))

кластер 1 отправил 129.24324000000055 биткойнов в кластер 2


In [32]:
finf_flow(from_to = (2, 1))

кластер 2 отправил 135.82000000000372 биткойнов в кластер 1


2

In [33]:
finf_flow(from_to = (1, 0))

кластер 1 отправил 300.55392887999795 биткойнов в кластер 0


In [34]:
finf_flow(from_to = (0, 1))

кластер 0 отправил 297.11968387999565 биткойнов в кластер 1


3

In [35]:
finf_flow(from_to = (2, 0))

кластер 2 отправил 4.869840000000001 биткойнов в кластер 0


In [36]:
finf_flow(from_to = (0, 2))

кластер 0 отправил 9.74013860999976 биткойнов в кластер 2


4

Функция, вычисляющая fee по кластерам

In [37]:
def find_fee(cluster: int):
    s = 0
    
    for unq_id in unique_transaction_ids:
        frame = address_stats[address_stats['transaction_id'] == unq_id]
        if cluster in frame[frame['sent'] > 0]['cluster'].values:
            s += (frame['sent'].sum() - frame['received'].sum()) / 10**8
            
    print('кластер {} потратил {} биткойнов на fee'.format(cluster, s))

In [38]:
find_fee(cluster = 1)

кластер 1 потратил 0.6671439999999648 биткойнов на fee


In [39]:
find_fee(cluster = 2)

кластер 2 потратил 0.7456672199999584 биткойнов на fee
